In [30]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = r'D:\github\localt20\processedData'

batting_data = pl.read_csv(os.path.join(directory, 'batting.csv'))
bowling_data = pl.read_csv(os.path.join(directory, 'bowling.csv'))
fielding_data = pl.read_csv(os.path.join(directory, 'fielding.csv'))
batting_data

Player,Season,Mat,Inns,Runs,SR,Ave,Country,Cum Ave,Cum Runs Total,Cum Inns Total,Cum SR,player_id
str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,str
"""A Andrews""","""2021/22""",3,1.0,0.0,0.0,0.0,"""Switzerland""",0.0,0.0,1.0,0.0,"""1d45c01a"""
"""A Andrews""","""2022""",4,3.0,34.0,85.0,17.0,"""Switzerland""",12.75,34.0,4.0,42.5,"""1d45c01a"""
"""A Ashok""","""2023""",1,0.0,0.0,0.0,0.0,"""New Zealand""",0.0,0.0,0.0,0.0,"""321be7e3"""
"""A Athanaze""","""2024""",4,4.0,75.0,122.95,25.0,"""West Indies""",25.0,75.0,4.0,122.95,"""58c2fac4"""
"""A Balbirnie""","""2015""",7,4.0,77.0,98.71,19.25,"""Ireland""",19.25,77.0,4.0,98.71,"""6ef13460"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zuhaib Zubair""","""2023/24""",2,1.0,13.0,144.44,13.0,"""United Arab Emirates""",13.0,13.0,1.0,144.44,"""9124aa4c"""
"""Zuhair Muhammad""","""2023/24""",2,2.0,7.0,43.75,3.5,"""Saudi Arabia""",3.5,7.0,2.0,43.75,"""22ae4973"""
"""Zulqarnain Haider""","""2019""",4,1.0,0.0,0.0,0.0,"""Spain""",0.0,0.0,1.0,0.0,"""2d46e8ed"""


# Checking Nans

In [31]:
dfs = [batting_data, bowling_data, fielding_data]

for df in dfs:
    print(df.shape)
    # Identify numeric columns
    numeric_columns = df.select(pl.col(pl.Float32) | pl.col(pl.Float64)).columns

    # Check for NaN values in numeric columns
    nan_check = df.select([
        pl.col(column).is_nan().any().alias(f"{column}_has_nan") for column in numeric_columns
    ])

    print(nan_check)

(10100, 13)
shape: (0, 0)
┌┐
╞╡
└┘
(10100, 15)
shape: (0, 0)
┌┐
╞╡
└┘
(10100, 10)
shape: (0, 0)
┌┐
╞╡
└┘


In [32]:
player_data = batting_data.join(bowling_data, on=['Player',"Country","Season"], how='inner')
# Player	Season	Mat	Inns	Runs	SR	Ave	Country	Cum Ave	Cum Runs Total	Cum Inns Total	Cum SR	player_id Overs	Wkts	Econ	Cumulative Overs	Cumulative Wkts	Cumulative Runs	Cumulative Inns	Cumulative Econ
player_data = player_data.select(["player_id","Player","Country","Season","Mat","Inns","Runs","SR","Ave","Cum Ave","Cum Runs Total","Cum Inns Total","Cum SR","Overs","Wkts","Econ","Cumulative Overs","Cumulative Wkts","Cumulative Runs","Cumulative Inns","Cumulative Econ"])
player_data

player_id,Player,Country,Season,Mat,Inns,Runs,SR,Ave,Cum Ave,Cum Runs Total,Cum Inns Total,Cum SR,Overs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ
str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1d45c01a""","""A Andrews""","""Switzerland""","""2021/22""",3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,4.0,6.1,10.0,4.0,61.0,3.0,20.333333
"""1d45c01a""","""A Andrews""","""Switzerland""","""2022""",4,3.0,34.0,85.0,17.0,12.75,34.0,4.0,42.5,1.0,0.0,21.0,11.0,4.0,82.0,4.0,20.5
"""321be7e3""","""A Ashok""","""New Zealand""","""2023""",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,7.0,4.0,1.0,28.0,1.0,28.0
"""58c2fac4""","""A Athanaze""","""West Indies""","""2024""",4,4.0,75.0,122.95,25.0,25.0,75.0,4.0,122.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""6ef13460""","""A Balbirnie""","""Ireland""","""2015""",7,4.0,77.0,98.71,19.25,19.25,77.0,4.0,98.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9124aa4c""","""Zuhaib Zubair""","""United Arab Emirates""","""2023/24""",2,1.0,13.0,144.44,13.0,13.0,13.0,1.0,144.44,6.0,0.0,8.0,6.0,0.0,48.0,2.0,24.0
"""22ae4973""","""Zuhair Muhammad""","""Saudi Arabia""","""2023/24""",2,2.0,7.0,43.75,3.5,3.5,7.0,2.0,43.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""2d46e8ed""","""Zulqarnain Haider""","""Spain""","""2019""",4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,3.0,5.3,10.0,3.0,53.0,4.0,13.25


In [33]:
player_data

player_id,Player,Country,Season,Mat,Inns,Runs,SR,Ave,Cum Ave,Cum Runs Total,Cum Inns Total,Cum SR,Overs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ
str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1d45c01a""","""A Andrews""","""Switzerland""","""2021/22""",3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,4.0,6.1,10.0,4.0,61.0,3.0,20.333333
"""1d45c01a""","""A Andrews""","""Switzerland""","""2022""",4,3.0,34.0,85.0,17.0,12.75,34.0,4.0,42.5,1.0,0.0,21.0,11.0,4.0,82.0,4.0,20.5
"""321be7e3""","""A Ashok""","""New Zealand""","""2023""",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,7.0,4.0,1.0,28.0,1.0,28.0
"""58c2fac4""","""A Athanaze""","""West Indies""","""2024""",4,4.0,75.0,122.95,25.0,25.0,75.0,4.0,122.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""6ef13460""","""A Balbirnie""","""Ireland""","""2015""",7,4.0,77.0,98.71,19.25,19.25,77.0,4.0,98.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9124aa4c""","""Zuhaib Zubair""","""United Arab Emirates""","""2023/24""",2,1.0,13.0,144.44,13.0,13.0,13.0,1.0,144.44,6.0,0.0,8.0,6.0,0.0,48.0,2.0,24.0
"""22ae4973""","""Zuhair Muhammad""","""Saudi Arabia""","""2023/24""",2,2.0,7.0,43.75,3.5,3.5,7.0,2.0,43.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""2d46e8ed""","""Zulqarnain Haider""","""Spain""","""2019""",4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,3.0,5.3,10.0,3.0,53.0,4.0,13.25


In [34]:
player_data = player_data.join(fielding_data, on=['Player',"Country","Season"], how='inner')
# player_id	Player	Country	Season	Mat	Inns	Runs	SR	Ave	Cum Ave	Cum Runs Total	Cum Inns Total	Cum SR	Overs	Wkts	Econ	Cumulative Overs	Cumulative Wkts	Cumulative Runs	Cumulative Inns	Cumulative Econ Dis	Ct	St	D/I
player_data = player_data.select(["player_id","Player","Country","Season","Mat","Inns","Runs","SR","Ave","Cum Ave","Cum Runs Total","Cum Inns Total","Cum SR","Overs","Wkts","Econ","Cumulative Overs","Cumulative Wkts","Cumulative Runs","Cumulative Inns","Cumulative Econ","Dis","Ct","St","D/I"])
player_data


player_id,Player,Country,Season,Mat,Inns,Runs,SR,Ave,Cum Ave,Cum Runs Total,Cum Inns Total,Cum SR,Overs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1d45c01a""","""A Andrews""","""Switzerland""","""2021/22""",3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,4.0,6.1,10.0,4.0,61.0,3.0,20.333333,4.0,4.0,0.0,1.333333
"""1d45c01a""","""A Andrews""","""Switzerland""","""2022""",4,3.0,34.0,85.0,17.0,12.75,34.0,4.0,42.5,1.0,0.0,21.0,11.0,4.0,82.0,4.0,20.5,7.0,7.0,0.0,1.0
"""321be7e3""","""A Ashok""","""New Zealand""","""2023""",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,7.0,4.0,1.0,28.0,1.0,28.0,0.0,0.0,0.0,0.0
"""58c2fac4""","""A Athanaze""","""West Indies""","""2024""",4,4.0,75.0,122.95,25.0,25.0,75.0,4.0,122.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.5
"""6ef13460""","""A Balbirnie""","""Ireland""","""2015""",7,4.0,77.0,98.71,19.25,19.25,77.0,4.0,98.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9124aa4c""","""Zuhaib Zubair""","""United Arab Emirates""","""2023/24""",2,1.0,13.0,144.44,13.0,13.0,13.0,1.0,144.44,6.0,0.0,8.0,6.0,0.0,48.0,2.0,24.0,0.0,0.0,0.0,0.0
"""22ae4973""","""Zuhair Muhammad""","""Saudi Arabia""","""2023/24""",2,2.0,7.0,43.75,3.5,3.5,7.0,2.0,43.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""2d46e8ed""","""Zulqarnain Haider""","""Spain""","""2019""",4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,3.0,5.3,10.0,3.0,53.0,4.0,13.25,1.0,1.0,0.0,0.25


In [35]:
player_data.write_csv(r'D:\github\localt20\processedData\playerStats.csv')